In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import math
import torch_geometric
import geopandas as gpd
import pandas as pd
import osmnx as ox
import momepy
import fiona
import networkx as nx
from torch_geometric.data import Data, Batch
from torch_geometric.transforms import RandomLinkSplit, LocalDegreeProfile
# from 

edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data1 = Data(x=x, edge_index=edge_index)

edge_index2 = torch.tensor([[0, 1, 0, 2],
                           [1, 0, 2, 0]], dtype=torch.long)

data2 = Data(x=x, edge_index=edge_index2)

data_batch = Batch.from_data_list([data1, data2])
data_batch.edge_index

tensor([[0, 1, 1, 2, 3, 4, 3, 5],
        [1, 0, 2, 1, 4, 3, 5, 3]])

In [2]:
# Experiment to confirm that batching increments pos and neg edge indexes
t = RandomLinkSplit(num_val=0, num_test=0, is_undirected=True, split_labels=True)
t_data1, t_data2 = t(data1)[0], t(data2)[0]
data_batch = Batch.from_data_list([t_data1, t_data2])
data_batch.pos_edge_label_index, data_batch.neg_edge_label_index

(tensor([[0, 1, 3, 3],
         [1, 2, 5, 4]]),
 tensor([[0, 2, 4, 5],
         [2, 0, 5, 4]]))

In [5]:
# Using LDP before and after split

disjoint_ratio=0.2
neg_sampling_ratio=1/disjoint_ratio

x = torch.tensor([[-2], [-1], [0], [1], [2], [3]], dtype=torch.float)

edge_index2 = torch.tensor([[0, 1, 0, 2, 4, 5, 0, 3, 1, 5],
                            [1, 0, 2, 0, 5, 4, 3, 0, 5, 1]], dtype=torch.long)

data = Data(x=x, edge_index=edge_index2)

t = RandomLinkSplit(num_val=0, num_test=0, disjoint_train_ratio=disjoint_ratio,
                          neg_sampling_ratio=neg_sampling_ratio, split_labels=True, is_undirected=True)
print(data.num_nodes)
t_deg = LocalDegreeProfile()
t_data1 = t_deg(data)
print(t_data1.x)
t_data2 = t_deg(t(data)[0])
print(t_data2.x)

6
tensor([[-2.0000,  3.0000,  1.0000,  2.0000,  1.3333,  0.5774],
        [-1.0000,  2.0000,  2.0000,  3.0000,  2.5000,  0.7071],
        [ 0.0000,  1.0000,  3.0000,  3.0000,  3.0000,  0.0000],
        [ 1.0000,  1.0000,  3.0000,  3.0000,  3.0000,  0.0000],
        [ 2.0000,  1.0000,  2.0000,  2.0000,  2.0000,  0.0000],
        [ 3.0000,  2.0000,  1.0000,  2.0000,  1.5000,  0.7071]])
tensor([[-2.0000,  3.0000,  1.0000,  2.0000,  1.3333,  0.5774,  2.0000,  1.0000,
          2.0000,  1.5000,  0.7071],
        [-1.0000,  2.0000,  2.0000,  3.0000,  2.5000,  0.7071,  2.0000,  2.0000,
          2.0000,  2.0000,  0.0000],
        [ 0.0000,  1.0000,  3.0000,  3.0000,  3.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000],
        [ 1.0000,  1.0000,  3.0000,  3.0000,  3.0000,  0.0000,  1.0000,  2.0000,
          2.0000,  2.0000,  0.0000],
        [ 2.0000,  1.0000,  2.0000,  2.0000,  2.0000,  0.0000,  1.0000,  2.0000,
          2.0000,  2.0000,  0.0000],
        [ 3.0000,  2.00

In [ ]:
t = RandomLinkSplit(num_val=0, num_test=0, is_undirected=True, split_labels=True)
t_data1, t_data2 = t(data1)[0], t(data2)[0]
data_batch = Batch.from_data_list([t_data1, t_data2])
data_batch.pos_edge_label_index, data_batch.neg_edge_label_index

In [2]:
y = torch.tensor([0, 1, 1, 1, 0, 0, 1])
pred = torch.tensor([1, 1, 0, 1, 0, 1, 1])
from sklearn.metrics import f1_score, precision_score, recall_score

f1_score(y, pred), precision_score(y, pred), recall_score(y, pred)

(0.6666666666666665, 0.6, 0.75)

In [4]:
disjoint_ratio=0.2
neg_sampling_ratio=1/disjoint_ratio

x = torch.tensor([[-2], [-1], [0], [1], [2]], dtype=torch.float)

edge_index2 = torch.tensor([[0, 1, 0, 2, 4, 5, 0, 3, 1, 5],
                            [1, 0, 2, 0, 5, 4, 3, 0, 5, 1]], dtype=torch.long)

data = Data(x=x, edge_index=edge_index2)

t = RandomLinkSplit(num_val=0, num_test=0, disjoint_train_ratio=disjoint_ratio,
                          neg_sampling_ratio=neg_sampling_ratio, split_labels=True, is_undirected=True)
t_data1 = t(data)[0]
print(t_data1.edge_index)
print(t_data1.pos_edge_label)
print(t_data1.pos_edge_label_index)
print(t_data1.neg_edge_label)
print(t_data1.neg_edge_label_index)
for u, v in zip(t_data1.edge_index[0], t_data1.edge_index[1]):
    print(u, v)

tensor([[4, 0, 0, 1, 5, 2, 1, 5],
        [5, 2, 1, 5, 4, 0, 0, 1]])
tensor([1.])
tensor([[0],
        [3]])
tensor([0., 0., 0., 0., 0.])
tensor([[0, 1, 4, 1, 3],
        [4, 4, 0, 2, 2]])
tensor(4) tensor(5)
tensor(0) tensor(2)
tensor(0) tensor(1)
tensor(1) tensor(5)
tensor(5) tensor(4)
tensor(2) tensor(0)
tensor(1) tensor(0)
tensor(5) tensor(1)
